# 1 name_scope与variable_scope
在tensorflow中,有两个scope, 一个是name_scope一个是variable_scope

## 1.1 name_scope

In [1]:
import tensorflow as tf
with tf.name_scope("hello") as name_scope:
    arr1 = tf.get_variable("arr1", shape=[2,10],dtype=tf.float32)
    
    print('name_scope : %s' % name_scope)
    print('arr1 : %s' % arr1.name)
    print("scope_name:%s " % tf.get_variable_scope().original_name_scope)

name_scope : hello/
arr1 : arr1:0
scope_name: 


如果在程序中在调用一次arr1 = tf.get_variable("arr1", shape=[2,10],dtype=tf.float32),那么就会出现命名冲突

## 1.2 variable_scope

In [1]:
import tensorflow as tf
with tf.variable_scope("hello") as variable_scope:
    arr1 = tf.get_variable("arr1", shape=[2, 10], dtype=tf.float32)

    print(variable_scope)
    print(variable_scope.name)
    print(arr1.name)
    print(tf.get_variable_scope().original_name_scope)
    #tf.get_variable_scope() 获取的就是variable_scope

    with tf.variable_scope("xixi") as v_scope2:
        print(tf.get_variable_scope().original_name_scope)
        #tf.get_variable_scope() 获取的就是v _scope2

hello
hello/arr1:0
hello/
hello/xixi/


## 1.3 比较

In [2]:
import tensorflow as tf
with tf.name_scope("name1"):
    with tf.variable_scope("var1"):
        w = tf.get_variable("w",shape=[2])
        res = tf.add(w,[3])

print(w.name)
print(res.name)

var1/w:0
name1/var1/Add:0


- name_scope对get_variable()创建的变量的名字不会有任何影响,而创建的op会被加上前缀.
- tf.get_variable_scope()返回的只是variable_scope,不管name_scope.所以以后我们在使用tf.get_variable_scope().reuse_variables()时可以无视name_scope
- name_scope是给op_name加前缀,variable_scope是给get_variable()创建的变量的名字加前缀
- 使用tf.Variable()的时候，tf.name_scope()和tf.variable_scope()都会给Variable和op的name属性加上前缀。 
- 使用tf.get_variable()的时候,tf.name_scope()就不会给tf.get_variable()创建出来的Variable加前缀。

## 1.4 共享变量
共享变量的前提是，变量的名字是一样的，变量的名字是由变量名和其scope前缀一起构成.tf.get_variable_scope().reuse_variables() 是允许共享当前scope下的所有变量

In [ ]:
import tensorflow as tf

with tf.variable_scope("1.4"):
    with tf.variable_scope("level1"):
        tf.get_variable("w",shape=[1])
        scope = tf.get_variable_scope()
        with tf.variable_scope("level2"):
            tf.get_variable("w", shape=[1])

    with tf.variable_scope("level1", reuse=True): #即使嵌套的variable_scope也会被reuse
        tf.get_variable("w",shape=[1])
        scope = tf.get_variable_scope()
        with tf.variable_scope("level2"):
            tf.get_variable("w", shape=[1])

ValueError: Variable 1.4/level1/level2/w does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?

#2 学习率衰减
- 指数衰减:exponential_decay, $D_i=D*rate^{\frac{global-step}{decay-step}}$
- 多项式衰减:polynomial_decay
- 自然指数衰减:natural_exp_decay
- 时间倒数衰减:inverse_time_decay